In [2]:
import numpy as np

def WaitingTimes(n, lam, mu):
    """
    Simulate waiting times for n customers in a single-server queue.
    
    Parameters:
    n: number of customers
    lam: arrival rate (lambda)
    mu: service rate (mu)
    
    Returns:
    Array of waiting times for each customer
    """
    # Generate interarrival times and service times
    T = np.random.exponential(1/lam, n)  # Interarrival times
    # Creates array of n random samples from Exp(λ)
    # np.random.exponential takes SCALE parameter = 1/rate
    # So for rate λ, we pass 1/λ
    
    V = np.random.exponential(1/mu, n)   # Service times
    # Creates array of n random samples from Exp(μ)
    # For rate μ, we pass 1/μ
    
    # Initialize arrays
    A = np.zeros(n)  # Arrival times
    # Creates array of n zeros to store arrival time of each customer
    
    D = np.zeros(n)  # Departure times
    # Creates array of n zeros to store departure time of each customer
    
    W = np.zeros(n)  # Waiting times
    # Creates array of n zeros to store waiting time of each customer
    
    # First customer
    A[0] = T[0]
    # Customer 1 arrives at time T[0] (first interarrival time from time 0)
    
    D[0] = A[0] + V[0]
    # Customer 1 departs at arrival time + service time
    # No waiting since queue starts empty
    
    W[0] = 0
    # Customer 1 has zero waiting time (given in problem)
    
    # Remaining customers
    for i in range(1, n):
        # Loop through customers 2 through n (indices 1 through n-1)
        
        A[i] = A[i-1] + T[i]
        # Customer i arrives T[i] time units after customer i-1
        # This implements: A_i = A_{i-1} + T_i
        
        D[i] = V[i] + max(A[i], D[i-1])
        # Customer i departs at: service time + max(arrival, previous departure)
        # If A[i] > D[i-1]: server idle, depart at A[i] + V[i]
        # If A[i] <= D[i-1]: server busy, depart at D[i-1] + V[i]
        # This implements: D_i = V_i + max(A_i, D_{i-1})
        
        W[i] = max(D[i-1] - A[i], 0)
        # Waiting time = time until previous customer finishes - arrival time
        # If D[i-1] > A[i]: must wait D[i-1] - A[i]
        # If D[i-1] <= A[i]: server idle, no wait (0)
        # This implements: W_i = max(D_{i-1} - A_i, 0)
    
    return W
    # Return the array of all waiting times

# Test with n=10, lambda=1, mu=1
np.random.seed(42)
# Set random seed for reproducibility
# Same seed = same "random" numbers every time

waiting_times = WaitingTimes(10, 1, 1)
# Call function to simulate 10 customers with λ=1, μ=1

print("Waiting times for n=10, λ=1, μ=1:")
print(waiting_times)
# Display the array of 10 waiting times

print(f"\nMean waiting time: {np.mean(waiting_times):.4f}")
# Compute and display average waiting time
# np.mean() computes average of array
# :.4f formats to 4 decimal places

Waiting times for n=10, λ=1, μ=1:
[0.         0.         2.18681178 3.06029877 3.12936153 3.16044422
 3.30321688 1.65473974 1.47958542 0.81387242]

Mean waiting time: 1.8788


#### Question 1d.
$$
\begin{align*}
E[W_2] &= \int_0^\infty P(W_2 \geq c) \, dc \quad \text{(survival function formula)} \\
\\
&= \int_0^\infty \frac{\lambda}{\lambda + \mu} e^{-\mu c} \, dc \quad \text{(from part (a))} \\
\\
&= \frac{\lambda}{\lambda + \mu} \int_0^\infty e^{-\mu c} \, dc \\
\\
&= \frac{\lambda}{\lambda + \mu} \left[\frac{-1}{\mu} e^{-\mu c}\right]_0^\infty \\
\\
&= \frac{\lambda}{\lambda + \mu} \left(0 - \frac{-1}{\mu}\right) \\
\\
&= \frac{\lambda}{\lambda + \mu} \cdot \frac{1}{\mu} \\
\\
&= \frac{\lambda}{\mu(\lambda + \mu)} \\
\\
\text{With } \lambda &= 1, \mu = 1: \\
E[W_2] &= \frac{1}{1 \cdot (1 + 1)} = \frac{1}{2} = 0.5
\end{align*}
$$

In [7]:
# ============================================================================
# Estimating E[W_2] and P(W_2 > 1) using Monte Carlo
# ============================================================================

# Set parameters
lam = 1  # Arrival rate λ
mu = 1   # Service rate μ
n_simulations = 1000000  # Number of Monte Carlo simulations

# Initialize list to store W_2 samples
W2_samples = []

# Run Monte Carlo simulations
for _ in range(n_simulations):
    # Generate waiting times for first 3 customers
    # We need 3 customers to get W_0, W_1, W_2
    W = WaitingTimes(2, lam, mu)
    
    # Extract W_2 (the waiting time of the 2nd customer)
    # Remember: W[0] = W_1, W[1] = W_2, W[2] = W_3
    W2_samples.append(W[1])

# Convert list to numpy array for easier computation
W2_samples = np.array(W2_samples)

# Estimate E[W_2] using sample mean
# By Law of Large Numbers: sample mean → true mean as n → ∞
E_W2_mc = np.mean(W2_samples)

# Estimate P(W_2 > 1) using indicator function
# np.mean(W2_samples > 1) computes the fraction of samples where W_2 > 1
# This is equivalent to: (number of times W_2 > 1) / (total simulations)
P_W2_gt_1_mc = np.mean(W2_samples > 1)

# Compute theoretical values from part (a)
# E[W_2] = λ / (μ(λ + μ))
E_W2_theory = lam / (mu * (lam + mu))

# P(W_2 > 1) = (λ/(λ+μ)) * e^(-μ*c) with c = 1
P_W2_gt_1_theory = (lam / (lam + mu)) * np.exp(-mu * 1)

# Display results
print("="*60)
print("Estimating W_2 with λ=1, μ=1")
print("="*60)

print(f"\nE[W_2]:")
print(f"  Monte Carlo estimate: {E_W2_mc:.4f}")
print(f"  Theoretical value:    {E_W2_theory:.4f}")
print(f"  Absolute error:       {abs(E_W2_mc - E_W2_theory):.4f}")

print(f"\nP(W_2 > 1):")
print(f"  Monte Carlo estimate: {P_W2_gt_1_mc:.4f}")
print(f"  Theoretical value:    {P_W2_gt_1_theory:.4f}")
print(f"  Absolute error:       {abs(P_W2_gt_1_mc - P_W2_gt_1_theory):.4f}")

Estimating W_2 with λ=1, μ=1

E[W_2]:
  Monte Carlo estimate: 0.4998
  Theoretical value:    0.5000
  Absolute error:       0.0002

P(W_2 > 1):
  Monte Carlo estimate: 0.1839
  Theoretical value:    0.1839
  Absolute error:       0.0001


In [8]:
# ============================================================================
# Estimating E[W_100] using Monte Carlo
# ============================================================================

# Set parameters (same as before)
lam = 1  # Arrival rate λ
mu = 1   # Service rate μ
n_simulations = 100000  # Number of Monte Carlo simulations

# Initialize list to store W_100 samples
W100_samples = []

# Run Monte Carlo simulations
for _ in range(n_simulations):
    # Generate waiting times for first 101 customers
    # We need 101 customers to get W_0, W_1, ..., W_99, W_100
    W = WaitingTimes(100, lam, mu)
    
    # Extract W_100 (the waiting time of the 100th customer)
    # Remember: W[0] = W_1, W[1] = W_2, ..., W[99] = W_100
    W100_samples.append(W[99])

# Convert list to numpy array
W100_samples = np.array(W100_samples)

# Estimate E[W_100] using sample mean
# By Law of Large Numbers: sample mean → true mean as n → ∞
E_W100_mc = np.mean(W100_samples)

# Note: There is no simple closed-form theoretical value for E[W_100]
# For λ = μ (our case), the system is at the boundary of stability
# The queue is critically loaded (utilization ρ = λ/μ = 1)
# In steady-state (as i → ∞), waiting times would grow unbounded
# But for finite i = 100, we can estimate via simulation

# Display results
print("\n" + "="*60)
print("Estimating W_100 with λ=1, μ=1")
print("="*60)

print(f"\nE[W_100]:")
print(f"  Monte Carlo estimate: {E_W100_mc:.4f}")
print(f"  (No simple closed-form theoretical value available)")

# Optional: Compute some additional statistics
print(f"\nAdditional statistics for W_100:")
print(f"  Standard deviation:   {np.std(W100_samples):.4f}")
print(f"  Median:               {np.median(W100_samples):.4f}")
print(f"  95th percentile:      {np.percentile(W100_samples, 95):.4f}")


Estimating W_100 with λ=1, μ=1

E[W_100]:
  Monte Carlo estimate: 10.2527
  (No simple closed-form theoretical value available)

Additional statistics for W_100:
  Standard deviation:   8.4487
  Median:               8.4726
  95th percentile:      26.5015
